# CS 446 Final project | Residue interaction networks in protein structures

## Setup

Procure the data using a set of proteins and running them through the tool RING 2.0 (one such result is available at: http://old.protein.bio.unipd.it/ring/results/5fbf3a94d794dd1d8c5eab12)

### Import packages

In [ ]:
# installs, imports
!apt-get install libcairo2-dev libjpeg-dev libgif-dev
!pip install pycairo
# use pip to install python-igraph
!pip install python-igraph
import gzip, operator, timeit, urllib.request, collections, pandas as pd, csv, cairo, igraph # add the other packages
import numpy as np
import matplotlib.pyplot as plt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
The following additional packages will be installed:
  libcairo-script-interpreter2 libpixman-1-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc
The following NEW packages will be installed:
  libcairo-script-interpreter2 libcairo2-dev libgif-dev libpixman-1-dev
  libxcb-shm0-dev
0 upgraded, 5 newly installed, 0 to remove and 14 not upgraded.
Need to get 951 kB of archives.
After this operation, 4,084 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcairo-script-interpreter2 amd64 1.15.10-2ubuntu0.1 [53.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libpixman-1-dev amd64 0.34.0-2 [244 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxcb-shm0-dev amd64 1.13-2~ubuntu18.04 [6,684 B]
Get:4 http:/

### Pandas Configuration

In [ ]:
pd.set_option("display.precision", 3)

### Import datasets

In [ ]:
# upload data from Drive database
#path = "drive/MyDrive/CS446_final/datasets/structural/hb_sifs"
path = "hb_sifs_reduced"

In [ ]:
#!wget https://github.com/picodase/cs446_final/blob/main/hb_sifs.zip?raw=true -O hb_sifs.zip
!wget https://github.com/picodase/cs446_final/blob/main/hb_sifs_reduced.zip?raw=true -O hb_sifs_reduced.zip
!unzip hb_sifs_reduced.zip

--2020-12-08 22:10:38--  https://github.com/picodase/cs446_final/blob/main/hb_sifs_reduced.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/picodase/cs446_final/raw/main/hb_sifs_reduced.zip [following]
--2020-12-08 22:10:39--  https://github.com/picodase/cs446_final/raw/main/hb_sifs_reduced.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/picodase/cs446_final/main/hb_sifs_reduced.zip [following]
--2020-12-08 22:10:39--  https://raw.githubusercontent.com/picodase/cs446_final/main/hb_sifs_reduced.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent

Import the data by constructing a RIN from the sif file

In [ ]:
import os

rin_dfs = []
fileNames = []
for file in os.scandir(path):
    if (file.path.endswith(".sif") and file.is_file()):
      #print(file.name.replace(".sif",""))
      fileNames.append(file.name.split(".")[0])
      rin_dfs.append(pd.read_csv(file, sep='\t', names=["resA", "interacType","resB"]))

### Define partitioning scheme for each subnetwork type

In [ ]:
# partitioning scheme for each network type
hbond = {"HBOND:MC_MC", "HBOND:SC_MC", "HBOND:MC_SC", "HBOND:SC_SC"}
vdw = {"VDW:SC_SC", "VDW:MC_SC", "VDW:SC_MC", "VDW:LIG_SC"}
lig = {"IAC:LIG_SC","IAC:LIG_MC","VDW:LIG_SC"}
ππ = {"PIPISTACK:SC_SC"}

### Organism name dictionary

In [ ]:
common_names = {
    "3A59":"Ostrich",
    "3A0G":"Guinea pig",
    "1FHJ":"Maned wolf",
    "4HRR":"Arcid clam",
    "3BJ1":"met-Perch",
    "3AT5":"Side-necked turtle",
    "1HBR":"Chicken",
    "1HDS":"Sickling deer",
    "2RAO":"Human Factor X",
    "2QU0":"Sheep",
    "1GCV":"Mussel",
    "1OUT":"Trout",
    "1V4U":"Bluefin tuna",
    "3BCQ":"Red-tailed Brycon",
    "1G0B":"Horse",
    "1FSX":"Cow",
    "1SPG":"Teleost fish",
    "1CG5":"Cartilagenous fish",
    "1HBH":"Antarctic fish",
    "1FAW":"Graylag goose",
    "1LA6":"Inshore Antarctic fish"
}

In [ ]:
class_names = {
    "3A59":"Bird",
    "3A0G":"Mammal",
    "1FHJ":"Mammal",
    "4HRR":"Crustacean",
    "3BJ1":"Fish",
    "3AT5":"Turtle?",
    "1HBR":"Bird",
    "1HDS":"Mammal",
    "2RAO":"Human",
    "2QU0":"Mammal",
    "1GCV":"Crustacean",
    "1OUT":"Fish",
    "1V4U":"Fish",
    "3BCQ":"Fish",
    "1G0B":"Mammal",
    "1FSX":"Mammal",
    "1SPG":"Fish",
    "1CG5":"Fish",
    "1HBH":"Fish",
    "1FAW":"Bird",
    "1LA6":"Fish"
}

### Create edge-list and graph representation, obtain network parameters for each network



Subset the network for these types of interactions

In [ ]:
# create lists to hold each set of values for the specified interaction types
interac_cts = []
edgelists = []
graphs = []
clusters = []
cluster_sizes = []
diams = []
#d_norm = []
thr_motifs = []
four_motifs = []

for df in rin_dfs:
  # interaction counts
  interac_cts.append(df.interacType.value_counts())

  # interaction types
  i = {}
  i["tot"] = df[['resA','resB']].values.tolist()
  i["hbd"] = df[df.interacType.isin(hbond)][['resA','resB']].values.tolist()
  i["vdw"] = df[df.interacType.isin(vdw)][['resA','resB']].values.tolist()
  i["lig"] = df[df.interacType.isin(lig)][['resA','resB']].values.tolist()
  i["ππ"] = df[df.interacType.isin(ππ)][['resA','resB']].values.tolist()
  edgelists.append(i)

  # graphs
  g = {}
  g["tot"] = igraph.Graph.TupleList(i["tot"])
  g["hbd"] = igraph.Graph.TupleList(i["hbd"], directed=True)
  g["vdw"] = igraph.Graph.TupleList(i["vdw"])
  g["lig"] = igraph.Graph.TupleList(i["lig"])
  g["ππ"] = igraph.Graph.TupleList(i["ππ"])
  graphs.append(g)

  # clusters
  c = {}
  c["tot"] = g["tot"].clusters()
  c["hbd"] = g["hbd"].clusters()    
  c["vdw"] = g["vdw"].clusters()
  c["lig"] = g["lig"].clusters()
  c["ππ"] = g["ππ"].clusters()
  clusters.append(c)

  # cluster sizes
  c_s = {}
  c_s["tot"] = c["tot"].sizes()
  c_s["hbd"] = c["hbd"].sizes()
  c_s["vdw"] = c["vdw"].sizes()
  c_s["lig"] = c["lig"].sizes()
  c_s["ππ"] = c["ππ"].sizes()
  cluster_sizes.append(c_s)

  # diameters
  d = {}
  d["tot"] = g["tot"].diameter()
  d["hbd"] = g["hbd"].diameter()
  d["vdw"] = g["vdw"].diameter()
  d["lig"] = g["lig"].diameter()
  d["ππ"] = g["ππ"].diameter()
  diams.append(d)

  # diameter normalized to number of vertices in the network
  #d_n = {}

  #d_n["hbd"] = (d["hbd"]/len(g["hbd"].vs) if (len(g["hbd"].vs) != 0) else np.nan)
  #d_n["vdW"] = (d["vdw"]/len(g["vdw"].vs) if (len(g["vdw"].vs) != 0) else np.nan)
  #d_n["lig"] = (d["lig"]/len(g["lig"].vs) if (len(g["lig"].vs) != 0) else np.nan)
  #d_n["ππ"] = (d["ππ"]/len(g["ππ"].vs) if (len(g["ππ"].vs) != 0) else np.nan)
  #d_norm.append(d)

  # motifs
  # calculate motifs for each graph

  # three-motifs
  t_m = {}
  t_m["tot"] = g["tot"].motifs_randesu()
  t_m["hbd"] = g["hbd"].motifs_randesu()
  t_m["vdw"] = g["vdw"].motifs_randesu()
  t_m["lig"] = g["lig"].motifs_randesu()
  t_m["ππ"] = g["ππ"].motifs_randesu()
  thr_motifs.append(t_m)

  # four-motifs
  f_m = {}
  f_m["tot"] = g["tot"].motifs_randesu(size=4)
  f_m["hbd"] = g["hbd"].motifs_randesu(size=4)
  f_m["vdw"] = g["vdw"].motifs_randesu(size=4)
  f_m["lig"] = g["lig"].motifs_randesu(size=4)
  f_m["ππ"] = g["ππ"].motifs_randesu(size=4)
  four_motifs.append(f_m)

### Vectors

For a proof-of-concept, take two hemoglobin proteins (say, the first and second graphs in the set), make their interaction counts into numpy arrays, then compute their cosine similarity. NO NORMALIZATION IS NEEDED because cosine is independent of magnitude!

#### Interaction counts

In [ ]:
ic1 = np.array(interac_cts[0])
ic2 = np.array(interac_cts[1])

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Getting an error because 1UC3 doesn't contain any values
ic1 = ic1.reshape(1,-1)
ic2 = ic2.reshape(1,-1)

cosine_similarity(ic1,ic2)

ValueError: ignored

#### Three-motif counts

In [ ]:
mt1 = np.array(thr_motifs[0]["hbd"])
mt2 = np.array(thr_motifs[1]["hbd"])

mt1[np.isnan(mt1)] = 0
mt2[np.isnan(mt2)] = 0

In [ ]:
mt1 = mt1.reshape(1,-1)
mt2 = mt2.reshape(1,-1)

cosine_similarity(mt1,mt2)

In [ ]:

# create a DF to contain all of these data, concatenating each network measure as a new column

def tmCosSim(n1, n2):
  nets = {"hbd","vdw","lig","ππ"}
  summary = []

  for net in nets:
  
    n1 = thr_motifs[0][net]
    n2 = thr_motifs[3][net]

    n1 = np.array(n1)
    n2 = np.array(n2)

    n1[np.isnan(n1)] = 0
    n2[np.isnan(n2)] = 0

    n1 = n1.reshape(1,-1)
    n2 = n2.reshape(1,-1)

    summary.append(cosine_similarity(n1,n2)[0][0])

  return summary

  print(tmCosSim(n1,n2))

In [ ]:
# calculate triple-motif cosine similarity between two graphs
def tmCosSim(g1:igraph.Graph, g2:igraph.Graph):
  n1 = np.array(g1.motifs_randesu(3)).reshape(1,-1)
  n2 = np.array(g2.motifs_randesu(3)).reshape(1,-1)

  n1[np.isnan(n1)] = 0
  n2[np.isnan(n2)] = 0

  return cosine_similarity(n1,n2)

#print(tmCosSim(graphs[0]["tot"],graphs[3]["tot"]))
print(tmCosSim(graphs[0]["hbd"],graphs[3]["hbd"]))
print(tmCosSim(graphs[0]["vdw"],graphs[3]["vdw"]))
print(tmCosSim(graphs[0]["lig"],graphs[3]["lig"]))
print(tmCosSim(graphs[0]["ππ"],graphs[3]["ππ"]))

In [ ]:
# calculate quadruple-motif cosine similarity between two graphs
def qmCosSim(g1:igraph.Graph, g2:igraph.Graph):
  n1 = np.array(g1.motifs_randesu(4)).reshape(1,-1)
  n2 = np.array(g2.motifs_randesu(4)).reshape(1,-1)

  n1[np.isnan(n1)] = 0
  n2[np.isnan(n2)] = 0

  return cosine_similarity(n1,n2)

#print(qmCosSim(graphs[0]["tot"],graphs[3]["tot"]))
print(qmCosSim(graphs[0]["hbd"],graphs[3]["hbd"]))
print(qmCosSim(graphs[0]["vdw"],graphs[3]["vdw"]))
print(qmCosSim(graphs[0]["lig"],graphs[3]["lig"]))
print(qmCosSim(graphs[0]["ππ"],graphs[3]["ππ"]))

In [ ]:
# calculate interaction counts cosine similarity between two graphs
def icCosSim(df1:pd.DataFrame, df2:pd.DataFrame):
  n1 = np.array(df1.interacType.value_counts()).reshape(1,-1)
  n2 = np.array(df2.interacType.value_counts()).reshape(1,-1)

  n1[np.isnan(n1)] = 0
  n2[np.isnan(n2)] = 0

  return cosine_similarity(n1,n2)

print(icCosSim(rin_dfs[0],rin_dfs[1]))

## **Cosine Correlation Matrices**

### Interaction Counts

In [ ]:
#Cosine Correlation Matrix for interac_cts
rin_interact_data = pd.DataFrame(interac_cts, index=fileNames).T

def dfCosSim(n1: np.ndarray, n2: np.ndarray):
  return cosine_similarity(n1.reshape(1,-1), n2.reshape(1,-1))

rn_interact_cts_corr = rin_interact_data.corr(dfCosSim)
rn_interact_cts_corr.style.background_gradient(cmap='coolwarm')

### Three Motif

In [ ]:
#Cosine Correlation Matrices for 3 motif

thr_motif_tot = [d['tot'] for d in thr_motifs]
thr_motif_hbd = [d['hbd'] for d in thr_motifs]
thr_motif_vdw = [d['vdw'] for d in thr_motifs]
thr_motif_lig = [d['lig'] for d in thr_motifs]
thr_motif_pipi = [d['ππ'] for d in thr_motifs]

rn_thrmotif_tot_data = pd.DataFrame(thr_motif_tot, index=fileNames).T
rn_thrmotif_hbd_data = pd.DataFrame(thr_motif_hbd, index=fileNames).T
rn_thrmotif_vdw_data = pd.DataFrame(thr_motif_vdw, index=fileNames).T
rn_thrmotif_lig_data = pd.DataFrame(thr_motif_lig, index=fileNames).T
rn_thrmotif_pipi_data = pd.DataFrame(thr_motif_pipi, index=fileNames).T

rn_thrmotif_tot_corr = rn_thrmotif_tot_data.corr(dfCosSim)
rn_thrmotif_hbd_corr = rn_thrmotif_hbd_data.corr(dfCosSim)
rn_thrmotif_vdw_corr = rn_thrmotif_vdw_data.corr(dfCosSim)
rn_thrmotif_lig_corr = rn_thrmotif_lig_data.corr(dfCosSim)
rn_thrmotif_pipi_corr = rn_thrmotif_pipi_data.corr(dfCosSim)

In [ ]:
#Setting axis = None fixes the color value issue
rn_thrmotif_tot_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
#Setting axis = None fixes the color value issue
rn_thrmotif_hbd_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
rn_thrmotif_vdw_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
rn_thrmotif_lig_corr.style.background_gradient(cmap='coolwarm', axis = None)

In [ ]:
rn_thrmotif_pipi_corr.style.background_gradient(cmap='coolwarm',axis=None)

### Four-motif

In [ ]:
#Cosine Correlation Matrices for 3 motif
four_motif_hbd = [d['hbd'] for d in four_motifs]
four_motif_vdw = [d['vdw'] for d in four_motifs]
four_motif_lig = [d['lig'] for d in four_motifs]
four_motif_pipi = [d['ππ'] for d in four_motifs]

rn_fourmotif_hbd_data = pd.DataFrame(four_motif_hbd, index=fileNames).T
rn_fourmotif_vdw_data = pd.DataFrame(four_motif_vdw, index=fileNames).T
rn_fourmotif_lig_data = pd.DataFrame(four_motif_lig, index=fileNames).T
rn_fourmotif_pipi_data = pd.DataFrame(four_motif_pipi, index=fileNames).T

rn_fourmotif_hbd_corr = rn_fourmotif_hbd_data.corr(dfCosSim)
rn_fourmotif_vdw_corr = rn_fourmotif_vdw_data.corr(dfCosSim)
rn_fourmotif_lig_corr = rn_fourmotif_lig_data.corr(dfCosSim)
rn_fourmotif_pipi_corr = rn_fourmotif_pipi_data.corr(dfCosSim)

In [ ]:
rn_fourmotif_hbd_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
rn_fourmotif_vdw_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
rn_fourmotif_lig_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
rn_fourmotif_pipi_corr.style.background_gradient(cmap='coolwarm',axis=None)

## **Hypothesis Testing**

In [ ]:
rn_thrmotif_pipi_corr.rename(columns=common_names, index=common_names).style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
f = plt.figure(figsize=(12, 10))
plt.matshow(rn_fourmotif_hbd_corr, fignum=f.number)
plt.xticks(range(rn_fourmotif_pipi_corr.shape[1]), rn_fourmotif_pipi_corr.columns, fontsize=12, rotation=45)
plt.yticks(range(rn_fourmotif_pipi_corr.shape[1]), rn_fourmotif_pipi_corr.columns, fontsize=12)
cb = plt.colorbar()
plt.title('Four-Motif HBD Correlation Matrix', fontsize=16, pad=20);

In [ ]:
f = plt.figure(figsize=(12, 10))
plt.matshow(rn_thrmotif_hbd_corr, fignum=f.number)
plt.xticks(range(rn_thrmotif_hbd_corr.shape[1]), rn_thrmotif_hbd_corr.columns, fontsize=12, rotation=45)
plt.yticks(range(rn_thrmotif_hbd_corr.shape[1]), rn_thrmotif_hbd_corr.columns, fontsize=12)
cb = plt.colorbar()
plt.title('Three Motif HBD Correlation Matrix', fontsize=16, pad=20);

In [ ]:
total_corr = (rn_interact_cts_corr + rn_thrmotif_hbd_corr + rn_thrmotif_lig_corr + rn_thrmotif_vdw_corr + rn_thrmotif_pipi_corr + rn_fourmotif_hbd_corr + rn_fourmotif_lig_corr + rn_fourmotif_pipi_corr + rn_fourmotif_vdw_corr) / 9
renamed_total_corr = total_corr.rename(columns=common_names, index=common_names)
#renamed_total_corr = pd.Categorical(df, categories=class_names)
renamed_total_corr.style.background_gradient(cmap='coolwarm',axis=None)

In [ ]:
# sort renamed_total_corr by correlation value to some column, pick "Horse"
dfn = renamed_total_corr.sort_values('Horse', ascending=False, axis=1).sort_values('Horse', ascending=False, axis=0)

In [ ]:
f = plt.figure(figsize=(12, 12))
n_dfn = dfn
plt.matshow(dfn, fignum=f.number)
plt.xticks(range(dfn.shape[1]), dfn.columns, fontsize=8, rotation=45)
plt.yticks(range(dfn.shape[1]), dfn.columns, fontsize=16)
cb = plt.colorbar()
plt.title('All-feature Correlation Matrix', fontsize=16, pad=50);

In [ ]:
f = plt.figure(figsize=(12, 12))
n_renamed_total_corr = renamed_total_corr
plt.matshow(n_renamed_total_corr, fignum=f.number)
plt.xticks(range(n_renamed_total_corr.shape[1]), n_renamed_total_corr.columns, fontsize=8, rotation=45)
plt.yticks(range(n_renamed_total_corr.shape[1]), n_renamed_total_corr.columns, fontsize=16)
cb = plt.colorbar()
plt.title('All-feature Correlation Matrix', fontsize=8, pad=50);

In [ ]:
#2 Sample T-Test
from scipy import stats

#Prints outcoems of manually caluclated difference in means & Welche's 2 Sample T-Test
def ttester(tdf):
    #Subset dataframes into their respective classes and compare all-feature correlation values (Can move into individual values if we wanted)
    tdf = tdf.rename(columns=class_names,index = class_names)
    cnms = ["Mammal","Fish","Bird","Crustacean","Human","Turtle?"]

    tdf = tdf.replace(1,np.NaN)

    interactions = {}
    ttests = {}

    #Check similarity differences between similar species vs. evolutionarily distant species

    for bspec in cnms:

      if bspec == "Human":
        break
      #Mean of Same Species interactions
      sm = tdf.loc[[bspec],[bspec]].mean().mean(axis=0)
      #Calculate Standard Deviation - Subract mean from each value, square the result, mean of the results, sqrt that
      ssd = np.sqrt(((tdf.loc[[bspec],[bspec]] - sm) ** 2).mean().mean(axis=0))
      # COULD ALSO USE np.nanstd(tdf.loc[["Mammal"],["Mammal"]].values.tolist())

      #For comparitive species
      for compsp in cnms:
        #Mean & STD of differenct species interaction
        dm = tdf.loc[[bspec],[compsp]].mean().mean(axis=0)
        dsd = np.sqrt(((tdf.loc[[bspec],[compsp]] - dm) ** 2).mean().mean(axis=0))

        #upper and lower bounds of means and stds
        ub = (sm+ssd) - (dm-dsd)
        lb = (sm-ssd) - (dm+dsd)

        #Key for run
        dk = bspec + " - " + compsp

        #Limits for differences in means

        if ub >= 0 and lb <= 0:
          interactions[dk] = 0
        else:
          interactions[dk] = 1

        #Check if Welches T-Test Agrees?
        a = tdf.loc[[bspec],[bspec]].values.tolist()
        b = tdf.loc[[bspec],[compsp]].values.tolist()
        a = [i for sub in a for i in sub]
        b = [j for sub in b for j in sub] 

        #compare means of similarities between similar species in a class vs similarities in species of another class
        ttests[dk] = stats.ttest_ind(a,b,equal_var = False,nan_policy = 'omit').pvalue


    return(interactions,ttests)

In [ ]:
#Check to see which interactions fail the t-test (You can also print out if the manual calculations - r1 - to determine that the difference can be zero)

r1,t1 = ttester(total_corr)
r2,t2 = ttester(rn_fourmotif_pipi_corr)
r4,t4 = ttester(rn_thrmotif_pipi_corr)
r3,t3 = ttester(rn_thrmotif_hbd_corr)


α = 0.05  # significance level

for k,v in t1.items():
  if v <= α:
    print(k,v)

print("--------")

for k,v in t2.items():
  if v <= α:
    print(k,v)

print("-----------")

for k,v in t3.items():
  if v <= α:
    print(k,v)


print("----------------")
for k,v in t4.items():
  if v <= α:
    print(k,v)

## TO DO:
- Implement similarity matrices for overall network
  - Interaction counts
  - 3-motifs
  - 4-motifs
- Finish presentation
- Practice presentation (< 9mins)
